In [290]:
import pandas as pd
import numpy as np
import configparser
import pymysql
import requests
import re
from tqdm import tqdm
import configparser
import openai
import os
import json

In [291]:
config=configparser.ConfigParser()
config.read('/home/ubuntu/infoedge/llama-recipes/examples/configs/config.ini')

['/home/ubuntu/infoedge/llama-recipes/examples/configs/config.ini']

In [508]:
mydb= pymysql.connect(
host=config['iimjobs_db']['hostname'],
user=config['iimjobs_db']['username'],
password=config['iimjobs_db']['password'],
db=config['iimjobs_db']['db_name']
)

mycursor = mydb.cursor()

In [509]:
mycursor = mydb.cursor()

 
query = "SELECT userid from usr_applied_jobs_info where jobid=1372174"

 # 1372184

mycursor.execute(query)

result = mycursor.fetchall()


In [510]:
udf = pd.DataFrame(result,columns=['userid'])

In [511]:
uid_list = udf.userid.values

In [512]:
uid_list

array([1478439, 1044553, 1354853, 1198562,  179817, 2728451, 1360902,
        412973])

In [513]:
from user_info import *

In [514]:
df = get_user_data_search_embed(uid_list)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 59283.45it/s]


In [515]:
df

,id,resume,keywords,current_designation,user_experience,professional_info,education_info
0,1478439,"123\nDheerajj Bhagat\nE-COMMERCE B2B Sales ,Se...",Sales Marketing Client Operations Strategy Onb...,Deputy Manager,9,"[{'id': 3235988, 'userId': None, 'designation'...","[{'id': 1695815, 'userId': 1478439, 'institute..."
1,1044553,Bhav Mishra\nTel : 9168383838\nE-Mail- bhavmis...,Digital Sales Strategic Sourcing Business_Deve...,Regional Business Manager- South,5,"[{'id': 3756502, 'userId': None, 'designation'...","[{'id': 1161783, 'userId': 1044553, 'institute..."
2,1354853,2014 Bachelor in Commerce\n(B.Com Graduate)\nG...,Sales Planning Negotiating Monitoring Analytic...,"Central GTM - Lifestyle, Elec and GM (Planning...",11,"[{'id': 1951210, 'userId': None, 'designation'...","[{'id': 1550603, 'userId': 1354853, 'institute..."
3,1198562,GAYATRI NAIDU PASUPULETI\nA Program Manager wi...,Brand Marketing Sales Retail CRM Research Supp...,Program Manager - Offline Business,5,"[{'id': 3878018, 'userId': None, 'designation'...","[{'id': 1958146, 'userId': 1198562, 'institute..."
4,179817,"Aashish Dhiman Male, Indian, 35 years, (Englis...",Strategy Analysis CEO Research FMCG Selling En...,Manager,8,"[{'id': 260633, 'userId': None, 'designation':...","[{'id': 9350, 'userId': 179817, 'institute': {..."
5,2728451,CA VIBHU AGGARWAL\nFINANCE &amp; ACCOUNTS\nRes...,TAX Finance Financial Accounting Planning GAAP...,Assistant Manager,7,"[{'id': 4121946, 'userId': None, 'designation'...","[{'id': 97634907, 'userId': 2728451, 'institut..."
6,1360902,EXTRA-CURRICULARS\n- Tata Crucible 2019 Region...,Strategy Sales Marketing MIS Corporate_Strateg...,"Head of Strategy, Sales and Marketing - Launch...",5,"[{'id': 1931405, 'userId': None, 'designation'...","[{'id': 1535411, 'userId': 1360902, 'institute..."
7,412973,EDUCATIONAL QUALIFICATIONS\nPGDM Indian Instit...,Marketing Strategy Sales HNI Research Analysis...,Account Manager,7,"[{'id': 3636943, 'userId': None, 'designation'...","[{'id': 272580, 'userId': 412973, 'institute':..."


In [516]:
# df[['id','resume']].sample(10).values

In [517]:
df

,id,resume,keywords,current_designation,user_experience,professional_info,education_info
0,1478439,"123\nDheerajj Bhagat\nE-COMMERCE B2B Sales ,Se...",Sales Marketing Client Operations Strategy Onb...,Deputy Manager,9,"[{'id': 3235988, 'userId': None, 'designation'...","[{'id': 1695815, 'userId': 1478439, 'institute..."
1,1044553,Bhav Mishra\nTel : 9168383838\nE-Mail- bhavmis...,Digital Sales Strategic Sourcing Business_Deve...,Regional Business Manager- South,5,"[{'id': 3756502, 'userId': None, 'designation'...","[{'id': 1161783, 'userId': 1044553, 'institute..."
2,1354853,2014 Bachelor in Commerce\n(B.Com Graduate)\nG...,Sales Planning Negotiating Monitoring Analytic...,"Central GTM - Lifestyle, Elec and GM (Planning...",11,"[{'id': 1951210, 'userId': None, 'designation'...","[{'id': 1550603, 'userId': 1354853, 'institute..."
3,1198562,GAYATRI NAIDU PASUPULETI\nA Program Manager wi...,Brand Marketing Sales Retail CRM Research Supp...,Program Manager - Offline Business,5,"[{'id': 3878018, 'userId': None, 'designation'...","[{'id': 1958146, 'userId': 1198562, 'institute..."
4,179817,"Aashish Dhiman Male, Indian, 35 years, (Englis...",Strategy Analysis CEO Research FMCG Selling En...,Manager,8,"[{'id': 260633, 'userId': None, 'designation':...","[{'id': 9350, 'userId': 179817, 'institute': {..."
5,2728451,CA VIBHU AGGARWAL\nFINANCE &amp; ACCOUNTS\nRes...,TAX Finance Financial Accounting Planning GAAP...,Assistant Manager,7,"[{'id': 4121946, 'userId': None, 'designation'...","[{'id': 97634907, 'userId': 2728451, 'institut..."
6,1360902,EXTRA-CURRICULARS\n- Tata Crucible 2019 Region...,Strategy Sales Marketing MIS Corporate_Strateg...,"Head of Strategy, Sales and Marketing - Launch...",5,"[{'id': 1931405, 'userId': None, 'designation'...","[{'id': 1535411, 'userId': 1360902, 'institute..."
7,412973,EDUCATIONAL QUALIFICATIONS\nPGDM Indian Instit...,Marketing Strategy Sales HNI Research Analysis...,Account Manager,7,"[{'id': 3636943, 'userId': None, 'designation'...","[{'id': 272580, 'userId': 412973, 'institute':..."


In [518]:
# df['id']=df['id'].astype(int)

In [519]:
df1= df[['id','resume']]

In [520]:
df1

,id,resume
0,1478439,"123\nDheerajj Bhagat\nE-COMMERCE B2B Sales ,Se..."
1,1044553,Bhav Mishra\nTel : 9168383838\nE-Mail- bhavmis...
2,1354853,2014 Bachelor in Commerce\n(B.Com Graduate)\nG...
3,1198562,GAYATRI NAIDU PASUPULETI\nA Program Manager wi...
4,179817,"Aashish Dhiman Male, Indian, 35 years, (Englis..."
5,2728451,CA VIBHU AGGARWAL\nFINANCE &amp; ACCOUNTS\nRes...
6,1360902,EXTRA-CURRICULARS\n- Tata Crucible 2019 Region...
7,412973,EDUCATIONAL QUALIFICATIONS\nPGDM Indian Instit...


In [521]:
# df1 = df1.sample()

In [522]:
documents = []
for x in df1:
    y = 'id: '+df1['id'].values+' '+df1['resume'].values
    documents.append(y)

In [523]:
len(documents[0])

8

In [524]:
# documents[0]

In [525]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter 
from langchain.vectorstores import FAISS


In [526]:
# documents = df1.values

In [527]:
# documents[90]

In [528]:
# documents[:, 0] = [{'id':int(value)} for value in documents[:, 0]]


In [529]:
# documents[0][90]

In [530]:
# len(documents)

In [531]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=1)
splits = text_splitter.split_text(str(documents[0]))
print(len(splits), splits[0])

105 ["id: 1478439 123\nDheerajj Bhagat\nE-COMMERCE B2B Sales ,Seller Onboarding ,Operations , Process , Vendor &amp;\nKey Account Management , fulfilment And E-commerce Logistics\nOperations, Management.\nAn enthusiastic &amp; high energy-driven professional, aiming to utilize the professional\nexperience, knowledge and skills to create a growth path for own self and the\norganization\nLocation Preference: PAN India\ndheerajjbhagat@gmail.com +91 98273-63662 , 79993-32017\nProfile Summary\nHighly


In [532]:
len(splits)

105

In [533]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',
                                       model_kwargs={'device': 'cuda'})

In [534]:
db = FAISS.from_texts(documents[0], embeddings)


In [535]:
import langchain
from queue import Queue
from typing import Any
from langchain.llms.huggingface_text_gen_inference import HuggingFaceTextGenInference
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.schema import LLMResult
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts.prompt import PromptTemplate
from anyio.from_thread import start_blocking_portal #For model callback streaming
import re
import gradio as gr
langchain.debug=True 

In [536]:
model = 'meta-llama/Llama-2-7b-chat-hf'

In [81]:
from langchain.llms import VLLM

llm = VLLM(model=model,
           trust_remote_code=True,  # mandatory for hf models
           max_new_tokens=128,
           top_k=10,
           top_p=0.95,
           temperature=0.6,
           # tensor_parallel_size=... # for distributed inference
)

INFO 02-07 15:53:07 llm_engine.py:73] Initializing an LLM engine with config: model='meta-llama/Llama-2-7b-chat-hf', tokenizer='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=auto, tensor_parallel_size=1, quantization=None, enforce_eager=False, seed=0)
INFO 02-07 15:54:10 llm_engine.py:223] # GPU blocks: 795, # CPU blocks: 512
INFO 02-07 15:54:13 model_runner.py:394] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 02-07 15:54:24 model_runner.py:437] Graph capturing finished in 11 secs.


In [575]:
template = """
[INST]Use the following pieces of context to answer the question. If no context provided, answer like a AI assistant.
{context}
Question: {question} [/INST]
"""

retriever = db.as_retriever(
        search_kwargs={"k": 1}
    )

In [576]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm, 
    retriever=retriever,     
    chain_type_kwargs={
        "prompt": PromptTemplate(
            template=template,
            input_variables=["context", "question"],
        ),
    }
)

In [577]:
result = qa_chain({"query": "give me all those ids who have worked as manager"})
# print(result)

[chain/start] [1:chain:RetrievalQA] Entering Chain run with input:
{
  "query": "give me all those ids who have worked as manager"
}
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain > 4:chain:LLMChain] Entering Chain run with input:
{
  "question": "give me all those ids who have worked as manager",
  "context": "id: 1360902 EXTRA-CURRICULARS\n- Tata Crucible 2019 Regional Finalist\n- Inducted Rotaract Club of PESU\n- Director of Public Relations - RCP\n- Organized Two Tech Workshops\n- Volunteer with Three college clubs\n- District Table-Tennis Silver Medalist\nPROFESSIONAL SKILLS\nProgram and Process Management\nP&amp;L Management\nBusiness Strategy and Planning\nLarge Team Handling\nGTM &amp; Growth Marketing\nAgile Methodologies\nPython, R Programming, SQL, SPSS\nTableau, PowerBI and dashboards\nWeb Analytics, Looker\nPredictive modeling, Regression\nData and MI

Processed prompts: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.27s/it]

[llm/end] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain > 4:chain:LLMChain > 5:llm:VLLM] [4.28s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Based on the provided resume, the following individuals have worked as managers:\n\n1. Karthik Narayan - Director of Public Relations, RCP; Project Manager, GrabOn Cricket Fantasy League; Associate Vice President, Business and Strategy, Populencer\n2. NoBroker.in - Led a team of 95, increased Launch vertical's gross Business Value from INR 18 Cr. to 700 Cr. in seven months; Designed the entire MIS and data-structures for analysis and reporting using prediction models, look-up",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/end] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain > 4:chain:LLMChain] [4.28s] Exiting Chain run with output:
{
  "text": "Based on the provided resume, the following individuals have worked as managers:

In [578]:
result

{'query': 'give me all those ids who have worked as manager',
 'result': "Based on the provided resume, the following individuals have worked as managers:\n\n1. Karthik Narayan - Director of Public Relations, RCP; Project Manager, GrabOn Cricket Fantasy League; Associate Vice President, Business and Strategy, Populencer\n2. NoBroker.in - Led a team of 95, increased Launch vertical's gross Business Value from INR 18 Cr. to 700 Cr. in seven months; Designed the entire MIS and data-structures for analysis and reporting using prediction models, look-up"}

In [579]:
user_id_pattern = r'id: (\d+)' and r'ID: (\d+)'

user_ids = re.findall(user_id_pattern, result['result'])

In [580]:
user_ids

[]

In [562]:
data = get_user_data_search_embed(user_ids)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 25653.24it/s]


In [565]:
data[['resume']].values

array([['Bhav Mishra\nTel : 9168383838\nE-Mail- bhavmishra1991@gmail.com\nAbout Me\nProven sales and account\nmanagement professional with\ndemonstrated communication skills.\nAnalytical, strategic planner and\naccomplished team player in tough\nmarkets. Detail and goal oriented\n,with 6+ years of experience across\nindustries. Able to handle and\ndeescalate high-stress situations.\nEmployment History\nAvanse Financial Services (Account Manager- Digital Business) JUL21-\nNOV22\n Account Management Driving Partner Relationships (B2B) and strategic alliances\nfor profitable engagement and co-ordinated with all key stakeholders external and\ninternal for a frictionless partner experience in execution of existing and new\npartnerships for the digital business vertical.\n Responsible to meet assigned business targets (~500 cr INR) for profitable sales\nvolume and strategic objectives with line of sign on P&amp;L from 4 assigned partner\naccounts.\n Business Development Sourcing new partners

In [564]:
user_id_pattern

'ID: (\\d+)'